In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import os

import pyOpenLPT as lpt
redirector = lpt.PythonStreamRedirector()

def print_dict(dict):
    for key in dict:
        print(key+":", dict[key])
    print('')

In [3]:
# create folder for output
os.makedirs("../../test/results/test_StereoMatch/", exist_ok=True)
os.makedirs("../../test/results/test_Shake", exist_ok=True)
os.makedirs("../../test/results/test_IPR/", exist_ok=True)
os.makedirs("../../test/results/test_PredField/", exist_ok=True)
os.makedirs("../../test/results/test_Track/", exist_ok=True)

### StereoMatch class

In [2]:
# load cam 
cam = []
intensity_max = []
useid_list = []
for i in range(4):
    file = '../../test/inputs/test_StereoMatch/cam' + str(i+1) + '.txt'
    cam.append(lpt.math.Camera(file))
    intensity_max.append(255)
    useid_list.append(i)
cam_list = lpt.math.CamList(cam, intensity_max, useid_list)
print_dict(cam_list.to_dict())

# load 2d tracer 
tr2d_list_all = []
for i in range(4):
    file = '../../test/inputs/test_StereoMatch/pt2d_list_cam' + str(i+1) + '.csv'
    pt2d = np.loadtxt(file, delimiter=',')
    n_tr2d = pt2d.shape[0]
    print('cam'+str(i+1)+':', n_tr2d, 'tracers')
    
    tr2d = []
    for j in range(n_tr2d):
        tr2d.append(lpt.object.Tracer2D(lpt.math.Pt2D(pt2d[j,0], pt2d[j,1])))
        
    tr2d_list_all.append(tr2d)
    

cam_list: [<pyOpenLPT.math.Camera object at 0x000001EA437A44F0>, <pyOpenLPT.math.Camera object at 0x000001EA437A4530>, <pyOpenLPT.math.Camera object at 0x000001EA437A4570>, <pyOpenLPT.math.Camera object at 0x000001EA437A45B0>]
intensity_max: [255, 255, 255, 255]
useid_list: [0, 1, 2, 3]
cam1: 32926 tracers
cam2: 32926 tracers
cam3: 32926 tracers
cam4: 32926 tracers


In [4]:
# initialize the stereomatch params 
param = lpt.stb.SMParam()
param.tor_2d = 1
param.tor_3d = 5e-3
param.n_thread = 6
param.check_id = 3
param.check_radius = 1
param.is_delete_ghost = False

print('StereoMatch Parameters:')
for key in param.to_dict().keys():
    print(key+':', param.to_dict()[key])
    

StereoMatch Parameters:
tor_2d: 1.0
tor_3d: 0.005
n_thread: 6
check_id: 3
check_radius: 1.0
is_delete_ghost: False
is_update_inner_var: False


In [5]:
# stereo match
sm = lpt.stb.StereoMatch(param, cam_list)
tr3d_list = sm.match(tr2d_list_all)

	Finish stereomatching: n_before_del = 34499.


In [7]:
pt3d = tr3d_list[0]._pt_center
pt3d.print(3)


Matrix = 
(dim_row, dim_col) = (3,1)
| -1.234e+01 |
| 4.884e+00 |
| -2.491e+00 |



In [6]:
sm.saveObjInfo("../../test/results/test_StereoMatch/tr3d.csv", tr3d_list)

### OTF class

In [5]:
otf = lpt.stb.OTF()
otf.loadParam("../../test/inputs/test_OTF/otf.txt")

param = otf._param 
print_dict(param.to_dict())

dx: 0.1111111111111111
dy: 0.1111111111111111
dz: 0.1111111111111111
n_cam: 4
nx: 10
ny: 10
nz: 10
n_grid: 1000
a: <pyOpenLPT.math.Matrix object at 0x00000214BD7DF430>
b: <pyOpenLPT.math.Matrix object at 0x00000214BD890F70>
c: <pyOpenLPT.math.Matrix object at 0x00000214BD8905B0>
alpha: <pyOpenLPT.math.Matrix object at 0x00000214BD8907B0>
boundary: <pyOpenLPT.AxisLimit object at 0x00000214BD890E70>
grid_x: [0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0]
grid_y: [0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0]
grid_z: [0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0]


In [9]:
# get otf param at (x,y,z)
cam_id = 0
pt3d = lpt.math.Pt3D(0.1, 0.5, 0.4)

otf_param = otf.getOTFParam(cam_id, pt3d)
print(otf_param)

[125.0, 1.5, 1.5, 0.0]


### Shake class

In [23]:
# load cam 
cam = []
intensity_max = []
useid_list = []
for i in range(4):
    file = '../../test/inputs/test_Shake/cam' + str(i+1) + '.txt'
    cam.append(lpt.math.Camera(file))
    intensity_max.append(255)
    useid_list.append(i)
cam_list = lpt.math.CamList(cam, intensity_max, useid_list)
# print_dict(cam_list.to_dict())

# load image path 
imgio_list = []
folder = '../../test/inputs/test_Shake/'
for i in range(4):
    file = 'cam' + str(i+1) + 'ImageNames.txt'
    imgio = lpt.math.ImageIO()
    imgio.loadImgPath(folder, file)
    # print(imgio.to_dict())
    
    imgio_list.append(imgio)
    
# load image 
img_list = []
for i in range(4):
    img_list.append(imgio_list[i].loadImg(0))

# find 2d tracer 
tr2d_list_all = []
property = [255, 30, 2]
for i in range(4):
    tr2d_list = lpt.object.ObjectFinder2D().findObject2D(img_list[i], property)
    tr2d_list_all.append(tr2d_list)
    print('cam'+str(i+1)+':', len(tr2d_list), 'tracers')


cam1: 11995 tracers
cam2: 12006 tracers
cam3: 11991 tracers
cam4: 11969 tracers


In [24]:
# stereo match
sm_param = lpt.stb.SMParam()
sm_param.tor_2d = 1.
sm_param.tor_3d = 2.4e-2
sm_param.n_thread = 6
sm_param.check_id = 3
sm_param.check_radius = 3
sm_param.is_delete_ghost = True
sm_param.is_update_inner_var = False

sm = lpt.stb.StereoMatch(sm_param, cam_list)
tr3d_list = sm.match(tr2d_list_all)
sm.saveObjInfo("../../test/results/test_Shake/tr3d_img_deleteGhost.csv", tr3d_list)

	Finish stereomatching: n_before_del = 14368.
	Finish deleting gohst match: n_del = 3559, n_after_del = 10809.


In [25]:
# otf 
boundary = lpt.AxisLimit()
boundary.x_min = -20
boundary.x_max =  20
boundary.y_min = -20
boundary.y_max =  20
boundary.z_min = -20
boundary.z_max =  20
otf = lpt.stb.OTF()
otf.loadParam(4, 2, 2, 2, boundary)

# shake
shake = lpt.stb.Shake(cam_list=cam_list, shake_width=0.01, score_min=0.1, n_loop=4, n_thread=6)
tr3d_list_shake = shake.runShake(tr3d_list, otf, img_list, False)

sm.saveObjInfo("../../test/results/test_Shake/tr3d_img_shake.csv", tr3d_list_shake)

	Shake::removeGhost: 0 ghost tracers are removed.


### IPR class

In [26]:
# load cam 
cam = []
intensity_max = []
useid_list = []
for i in range(4):
    file = '../../test/inputs/test_IPR/cam' + str(i+1) + '.txt'
    cam.append(lpt.math.Camera(file))
    intensity_max.append(255)
    useid_list.append(i)
cam_list = lpt.math.CamList(cam, intensity_max, useid_list)
# print_dict(cam_list.to_dict())

# load image path 
imgio_list = []
folder = '../../test/inputs/test_IPR/'
for i in range(4):
    file = 'cam' + str(i+1) + 'ImageNames.txt'
    imgio = lpt.math.ImageIO()
    imgio.loadImgPath(folder, file)
    # print(imgio.to_dict())
    
    imgio_list.append(imgio)
    
# load image 
img_list = []
for i in range(4):
    img_list.append(imgio_list[i].loadImg(0))

In [27]:
boundary = lpt.AxisLimit(-20, 20, -20, 20, -20, 20)
otf = lpt.stb.OTF(4, 2, 2, 2, boundary)
tr2d_properties = [255, 30, 2]

# IPR param
ipr_param = lpt.stb.IPRParam()
print('<IPR Parameters>')
print_dict(ipr_param.to_dict())

<IPR Parameters>
tri_only: False
n_thread: 6
n_loop_ipr: 4
n_loop_ipr_reduced: 2
n_obj2d_max: 100000
tol_2d: 1.0
tol_3d: 0.024
check_id: 3
check_radius: 3.0
n_loop_shake: 1
shake_width: 0.024
ghost_threshold: 0.1


In [28]:
# IPR
ipr = lpt.stb.IPR(cam_list, img_list, ipr_param)
tr3d_list = ipr.runIPR(tr2d_properties, otf, n_reduced=0)

ipr.saveObjInfo("../../test/results/test_IPR/tr3d_img_ipr.csv", tr3d_list)

	Number of found tracers in each camera: 11995,12006,11991,11969,
	Finish stereomatching: n_before_del = 14368.
	Finish deleting gohst match: n_del = 3559, n_after_del = 10809.
	Matching time = 1.566 [s]
	Shake::removeGhost: 0 ghost tracers are removed.
	Shake time = 0.203 [s]
  IPR step 0: find 10809 particles. 
	Number of found tracers in each camera: 1470,1468,1456,1479,
	Finish stereomatching: n_before_del = 1236.
	Finish deleting gohst match: n_del = 157, n_after_del = 1079.
	Matching time = 0.421 [s]
	Shake::removeGhost: 0 ghost tracers are removed.
	Shake time = 0.024 [s]
  IPR step 1: find 11888 particles. 
	Number of found tracers in each camera: 396,393,380,399,
	Finish stereomatching: n_before_del = 64.
	Finish deleting gohst match: n_del = 9, n_after_del = 55.
	Matching time = 0.355 [s]
	Shake::removeGhost: 0 ghost tracers are removed.
	Shake time = 0.008 [s]
  IPR step 2: find 11943 particles. 
	Number of found tracers in each camera: 341,337,327,344,
	Finish stereomatchin

### PredField class 

In [2]:
# load 3D points 
inputs_prev = np.loadtxt('../../test/inputs/test_PredField/pts_prev_1.csv', delimiter=',') 
inputs_curr = np.loadtxt('../../test/inputs/test_PredField/pts_curr_1.csv', delimiter=',')

pt3d_list_prev = []
pt3d_list_curr = []
n_pt = inputs_prev.shape[0]
for i in range(n_pt):
    pt3d_list_prev.append(lpt.math.Pt3D(inputs_prev[i,0], inputs_prev[i,1], inputs_prev[i,2]))
    pt3d_list_curr.append(lpt.math.Pt3D(inputs_curr[i,0], inputs_curr[i,1], inputs_curr[i,2]))

In [3]:
# PredField param 
pred_param = lpt.stb.PFParam()
pred_param.limit = lpt.AxisLimit(-20, 20, -20, 20, -20, 20)
pred_param.nx = 51
pred_param.ny = 51
pred_param.nz = 51
pred_param.r = 0.4 
pred_param.nBin_x = 5
pred_param.nBin_y = 5
pred_param.nBin_z = 5

# PredField
pf = lpt.stb.PredField(pred_param, pt3d_list_prev, pt3d_list_curr)

In [4]:
print('<PredField>')
print_dict(pf.to_dict())

<PredField>
_param: <pyOpenLPT.stb.PFParam object at 0x0000022B4C00EF30>
_disp_field: <pyOpenLPT.math.Matrix object at 0x0000022B2BE2E170>


### Track class

In [2]:
# initialize
track = lpt.stb.TracerTrack()
print('<TracerTrack>')
print_dict(track.to_dict())

# add obj to the track 
obj3d = lpt.object.Tracer3D(lpt.math.Pt3D(1, 2, 3))
obj3d._error = 4 
obj3d._n_2d = 1
obj3d._camid_list = [6]
obj3d._tr2d_list = [lpt.object.Tracer2D(lpt.math.Pt2D(7, 8))]

track.addNext(obj3d, 5)
print('<TracerTrack> after adding')
print_dict(track.to_dict())

<TracerTrack>
_obj3d_list: []
_t_list: []
_n_obj3d: 0
_active: True

<TracerTrack> after adding
_obj3d_list: [<pyOpenLPT.object.Tracer3D object at 0x00000191E82126F0>]
_t_list: [5]
_n_obj3d: 1
_active: True



In [3]:
n_cam_all = 10
fps = 1

file = "../../test/results/test_Track/test_function_1.csv"
with open(file, 'w') as f:
    f.write('TrackID,FrameID,WorldX,WorldY,WorldZ,Error,Ncam\n')
    for i in range(n_cam_all):
        f.write(',cam'+str(i)+'_x(col),cam'+str(i)+'_y(row)')
    f.write('\n')

track.saveTrack(file, 0, fps, n_cam_all)

### STB class 